In [ ]:
# Import libraries

import numpy as np
import pandas as pd
from IPython.display import display, HTML
from datetime import date, timedelta
import os
from matplotlib import pyplot as plt
import matplotlib.style as style
import matplotlib.dates as mdates
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.gridspec import GridSpec
from matplotlib.pylab import rcParams
import seaborn as sns
import locale
from google.oauth2 import service_account
from google.cloud import bigquery
import time
import unicodedata
import missingno as msno
import sweetviz as sv
from ydata_profiling import ProfileReport
from fuzzywuzzy import fuzz
from wordcloud import WordCloud
import requests
import random
import itertools
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from scipy import interpolate
from itertools import combinations
from plotly import graph_objects as go
import string
import pycountry
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Identify Excel files (within notebook's folder)

def excel_files():
    current_dir = os.getcwd()
    files = [file for file in os.listdir(current_dir) if file.endswith('.xlsx') or file.endswith('.csv')]

    print(f'folder name = {current_dir}\n')
    print('\033[1mExcel files within the folder :\033[0m')
    print(files)

In [ ]:
# Advanced value_counts

def value_counts_2(df, col_name, c_map = 'Blues', drop_null = False):
    
    x = df[col_name].value_counts(dropna = drop_null).reset_index()
    x.columns = [col_name.upper(), 'Count']
    x['% Total'] = (x['Count'] / x['Count'].sum()) * 100
    x.set_index(col_name.upper(), inplace = True)

    print(f"Total values = {x['Count'].sum():,}")
    
    return x.style.background_gradient(subset = '% Total', cmap = c_map).format(precision = 2)

#vc = value_counts_2(df,'status')
#vc

In [ ]:
# Datetime columns

def datetime_cols(df, datetime_col, datetime_format = 'YYYY-MM-DD', year = True, month = True, week = True):

    df[datetime_col] = pd.to_datetime(df[datetime_col], format = datetime_format)

    if year == True:
        df['year'] = pd.to_datetime(df[datetime_col]).dt.to_period('Y')
    if month == True:
        df['month'] = pd.to_datetime(df[datetime_col]).dt.to_period('M')
    if week == True:
        df['week'] = df[datetime_col].dt.to_period('W').apply(lambda x: x.start_time)
        df['week'] = df['week'].dt.strftime('%Y-%m-%d')

In [ ]:
# Consolidate Values

def consolidate_values(df, col_name, threshold = 1, other_value = 'Other', consolidate_null = True):

    a = df.copy()
    pop_counts = a[col_name].value_counts(dropna = not consolidate_null)
    total_pop = pop_counts.sum()
    threshold_value = total_pop * threshold / 100
    consolidated_values = pop_counts[pop_counts < threshold_value].index
    a[col_name] = a[col_name].apply(lambda x: other_value if x in consolidated_values else x)
    if consolidate_null:
        a[col_name] = a[col_name].fillna(other_value)

    return a

In [ ]:
# BigQuery connection

def query(query, project_name = 'rapyd-bq-poc-2020'):

    start_time = time.time()
    
    client = bigquery.Client(project = project_name)
    query_job = client.query(query)
    df = query_job.to_dataframe()

    seconds = round(time.time() - start_time, 0)
    print(f'Execution time = {str(timedelta(seconds = seconds))} minutes')
    print('------------------------------------')
    print(f'Rows = {df.shape[0]:,} | Columns = {df.shape[1]}'), print('')
    
    return df

In [ ]:
# Replace Accented Characters 

def replace_accents(text):
    return ''.join(char for char in unicodedata.normalize('NFD',str(text)) if not unicodedata.combining(char))

#df = df.applymap(replace_accents)

In [ ]:
# Advanced Crosstab

def advanced_crosstab(df, row, col, normalize_by = 'row', change = 'drop', chart_size = (10,6), \
                 cmap = 'Oranges', line = 'white', text = 8, rotation = 0):
    
    if normalize_by not in ['row', 'col', 'all']:
        raise ValueError(f'Expected "normalize_by" param to be either "row", "col" or "all". Got "{normalize_by}" instead.')
    if change not in ['abs_diff', 'drop']:
        raise ValueError(f'Expected "change" param to be either "abs_diff" or "drop". Got "{change}" instead.')

    df2 = df.copy()
    df2[row].fillna('-', inplace = True)
    df2[col].fillna('-', inplace = True)

    norm = {'row':0, 'col':1, 'all':True}.get(normalize_by)
    ct1 = pd.crosstab(df2[row], df2[col], normalize = norm)
    ct2 = pd.crosstab(df2[row], df2[col], margins = True, margins_name = 'Total')
    cross = ct1.combine_first(ct2)

    # push the total column & row to the end
    total_col = cross.pop('Total')
    cross['Total'] = total_col
    cross.sort_values(by = 'Total', ascending = False, inplace = True)

    total_row = cross.loc['Total']
    cross.drop('Total', axis = 0, inplace = True)
    cross.loc['Total'] = total_row

    if change == 'abs_diff':
        cross['absolute diff'] = cross['Total'].diff().fillna(0).astype(int)
        cross['absolute diff'].iloc[-1] = 0

    num = -1 if change == 'drop' else -2
    mask_1 = np.zeros(cross.shape)
    mask_1[:, num:] = True
    mask_1[-1, :] = True
    mask_2 = np.zeros(cross.shape)
    mask_2[:-1, :num] = True

    plt.figure(figsize = chart_size)
    plt.rcParams.update({'font.size':text})
    ax = sns.heatmap(cross,mask=mask_1,cbar=False,cmap=cmap,annot=True,fmt='.2%',annot_kws={'color':'black'},lw=2,linecolor=line)
    ax = sns.heatmap(cross,mask=mask_2,cbar=False,alpha=0,annot=True,fmt='.0f',annot_kws={'color':'black'},lw=2)  

    plt.xlabel(col.replace('_',' ').title()), plt.ylabel(row.replace('_',' ').title())
    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')
    ax.tick_params(axis = 'x', which = 'major', rotation = rotation)
    ax.tick_params(axis = 'y', which = 'major', rotation = rotation)
    ax.set_xlabel(col.replace('_',' ').title(), fontweight = 'bold', labelpad = 15)
    ax.set_ylabel(row.replace('_',' ').title(), fontweight = 'bold', labelpad = 15);

In [ ]:
# Send a Mail

import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 

csv_file = '/Users/itaymi/Desktop/Python/gmail_pass.csv'

def gmail(recipients, subject, email_message, files = [], csv_pass = csv_file):

    g = pd.read_csv(csv_file)
    sender, password = g.at[0,'gmail'], g.at[1,'gmail']

    msg = MIMEMultipart()  
    msg['From'] = sender  
    msg['To'] = recipients 
    msg['Subject'] = subject
    msg.attach(MIMEText(email_message, 'plain')) 

    for file in files:
        filename = file
        attachment = open(file, 'rb') 
        p1 = MIMEBase('application', 'octet-stream')  
        p1.set_payload((attachment).read()) 
        encoders.encode_base64(p1) 
        p1.add_header('Content-Disposition', 'attachment ; filename = %s' % filename) 
        msg.attach(p1) 

    server = smtplib.SMTP('smtp.gmail.com', 587) 
    server.starttls() 
    server.login(sender, password)
    server.sendmail(sender, recipients.split(','), msg.as_string())

    print('Mail sent successfully :)')
    
    server.quit() 

In [ ]:
# Missing values

def missing_values(df, fig_size = (16,8), w = [1,1], h = [1,1], ws = 0.3, hs = 0.8):

    print('\033[1mMissing values (%):\033[0m')
    print(round((df.isna().sum() / len(df)) * 100, 2))

    fig = plt.figure(figsize = fig_size)
    grid = GridSpec(2, 2, width_ratios = w, height_ratios = h, wspace = ws, hspace = hs)
    
    ax1 = plt.subplot(grid[0,0])
    msno.bar(df, fontsize = 8, color = 'dodgerblue', ax = ax1)
    ax1.set_title('Bar Chart (existing values)', fontsize = 10)
    
    ax2 = plt.subplot(grid[0,1])
    msno.heatmap(df, fontsize = 8, ax = ax2)
    ax2.set_title('Heatmap', fontsize = 10)
    
    ax3 = plt.subplot(grid[1,0])
    msno.dendrogram(df, fontsize = 8, ax = ax3)
    ax3.set_title('Dendrogram', fontsize = 10)
    
    ax4 = plt.subplot(grid[1,1])
    msno.matrix(df, fontsize = 8, ax = ax4)
    ax4.set_title('Matrix', fontsize = 10)
    
    fig.suptitle('Missing Data Patterns')
    fig.subplots_adjust(top = 0.85)
    plt.show()

In [ ]:
# Compare 2 dataframes

def compare_dataframes(df1,df2):
    
    def organize_df(df):
        df.sort_values(by = df.columns.tolist(), ignore_index = True, inplace = True)
        duplicates = df.duplicated(keep = False).astype(int)
        df['is_duplicate'] = duplicates
        
    organize_df(df1)
    organize_df(df2)
    
    # merged dataframe
    df3 = df1.iloc[:,:-1].merge(df2.iloc[:,:-1], how = 'outer')
    df3.drop_duplicates(ignore_index = True, inplace = True)
    
    # add df1 flag
    df3 = df3.merge(df1, how = 'left')
    df3.rename(columns = {'is_duplicate':'exists_df1'}, inplace = True)
    df3['exists_df1'] = df3['exists_df1'].apply(lambda x: 0 if pd.isna(x) else 1)
    
    # add df2 flag
    df3 = df3.merge(df2, how = 'left')
    df3.rename(columns = {'is_duplicate':'exists_df2'}, inplace = True)
    df3['exists_df2'] = df3['exists_df2'].apply(lambda x: 0 if pd.isna(x) else 1)
    
    # flag errors
    df3['error'] = df3.apply(lambda row: 0 if row['exists_df1']+row['exists_df2'] == 2 else 1, axis = 1)

    print(f'\033[1m1st df length = \033[0m{len(df1):,} (duplications = {df1.is_duplicate.sum():,})')
    print(f'\033[1m2nd df length = \033[0m{len(df2):,} (duplications = {df2.is_duplicate.sum():,})')
    print(f'\033[1mNo. of errors = \033[0m{df3.error.sum():,}')
    
    return df3

In [ ]:
# Cleaned number format

def format_fixer(df,cols):
    for col in cols:
        df[col] = df[col].apply(lambda x: ''.join([c if c.isdigit() or c == '.' else '' for c in str(x)]))
        df[col] = pd.to_numeric(df[col], errors = 'coerce')
    return df

In [ ]:
# Column to string

def column_to_string(df, col, values_type = 'str'):
    
    a = df.loc[:,[col]].copy()
    a.drop_duplicates(subset = col, inplace = True)
    
    if values_type == 'str':
        a[col] = "'" + a[col].astype(str) + "'"
    
    b = a[col].to_numpy()
    b = ','.join(str(x) for x in b) 
    
    return b

In [ ]:
def retention_matrix(df, 
                     cohort_col = 'trx_month', 
                     cohort_name = '1st transaction', 
                     cmap = 'Purples',
                     title_name = 'Retention Matrix', 
                     x_label = 'Months since 1st transaction', 
                     y_label = '1st transaction month',
                     figsize = (15,10), 
                     font_size = 8, 
                     months_to_exclude = 0, 
                     fmt = '.1%'):

    plt.rcParams.update({'figure.figsize':figsize, 'font.size':font_size})

    a = df.copy()

    # assign cohort   
    a[cohort_col] = pd.to_datetime(a[cohort_col])
    first = a.groupby('merchant_id', as_index = False)[cohort_col].min()
    first.columns = ['merchant_id','start']

    # retention dataframe
    retention = a.merge(first, on = 'merchant_id', how = 'inner')
    retention['period_diff'] = ((retention[cohort_col] - retention['start']) / np.timedelta64(1, 'M'))
    retention['period_diff'] = np.round(retention['period_diff'],0).fillna(-1).astype(int)
    
    # cohort size
    cohort_size = retention.groupby('start').merchant_id.nunique().sort_index(ascending = False).reset_index()
    diff = pd.crosstab(retention['start'], retention['period_diff']).apply(pd.to_numeric, errors = 'coerce').reset_index()

    # merge cohort size & retention dataframes
    x = cohort_size.merge(diff, on = 'start', how = 'left')
    x.rename(columns = {'start':cohort_name,'merchant_id':'Merchants'}, inplace = True)  

    # drop historical months
    if months_to_exclude > 0:
        x = x.iloc[:-months_to_exclude,:-months_to_exclude].copy()

    # add zero colomuns if needed (cubic dataframe)
    int_list = x.columns[2:].astype(int)
    for i, j in enumerate(x):
        if not i in int_list:
            x[i] = 0
    x = x.iloc[1:, :-2].copy()

    # columns reorder
    cols_1 = [cohort_name, 'Merchants']
    cols_2 = list(range(int(x.columns[2:].max()+1)))
    cols = cols_1 + cols_2
    x = x[cols]

    # convert to percentages
    cohort_sizes = x.loc[:,'Merchants']
    rate = x.iloc[:,1:].divide(cohort_sizes, axis = 0)
    rate = rate.iloc[:,1:].copy()
    rate.columns = rate.columns.astype(int)

    # merge to the final dataframe
    final = pd.concat([x.loc[:,cols_1], rate], axis = 1)
    final[cohort_name] = pd.to_datetime(final[cohort_name]).dt.strftime('%y-%b')
    final.drop(0, axis = 1, inplace = True)

    # reshape the dataframe to fit the heatmap visualization
    final.set_index(cohort_name, inplace = True)   
    mask_1 = np.zeros(final.shape)
    final[final == 0] = np.nan
    mask_1[:, 0] = True
    mask_2 = np.zeros(final.shape)
    mask_2[:, 1:] = True

    # visualization
    ax = sns.heatmap(final, mask = mask_1, cbar = False, cmap = cmap, alpha = 0.6, annot = True, fmt = fmt, lw = 2, annot_kws = {'color':'black'})
    ax = sns.heatmap(final, mask = mask_2, cbar = False, alpha = 0, annot = True, fmt = ',.0f', lw = 2, annot_kws = {'color':'black'})  

    # final touch ups
    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')
    ax.tick_params(axis = 'both', which = 'major', labelsize = font_size, rotation = 0)
    plt.title(title_name, weight = 'bold', pad = 15, fontsize = font_size + 6)
    plt.xlabel(x_label, labelpad = 18, fontsize = font_size + 2)
    plt.ylabel(y_label, labelpad = 18, fontsize = font_size + 2)
    plt.show()

    return final

In [ ]:
# Cartesian function

def add_cartesian(df, group_col = 'merchant_id', date_col = 'created_date', value_col = 'num_trx'):
    
    # add new columns
    a = df.copy()
    a[date_col] = pd.to_datetime(a[date_col])
    a['month'] = pd.to_datetime(a[date_col].dt.strftime('%Y-%m-01'))    
    a['cohort'] = pd.to_datetime(a.groupby(group_col)[date_col].transform(lambda x: x.min().replace(day = 1)))
    
    # months' dataframe
    months = pd.DataFrame(pd.date_range(start = a.month.min(), end = a.month.max(), freq = 'MS'), 
                          columns = ['month'])

    # monthly per user
    users = a.groupby([group_col,'month'], as_index = False)[value_col].sum()
    
    # Cartesian product of user_id & month
    cross = pd.MultiIndex.from_product([a[group_col].unique(), months.month.unique()], 
                                       names = [group_col,'month']).to_frame(index = False)

    # 1st merge
    x = cross.merge(users, on = [group_col,'month'], how = 'left')
    
    # 2nd merge (add cohort)
    x = x.merge(a.groupby(group_col, as_index = False).cohort.min(), on = group_col, how = 'inner')
    
    x[value_col] = x[value_col].fillna(0)
    x['month_diff'] = ((x['month'] - x['cohort']) / np.timedelta64(1,'M'))
    x['month_diff'] = np.round(x['month_diff']).fillna(-1).astype(int)
    
    # reorder columns
    x = x.loc[:,[group_col,'cohort','month','month_diff',value_col]].copy()
    
    # sort & drop redundant rows
    x = x[x.month >= x.cohort].copy()
    x.sort_values(by = [group_col,'month'], ignore_index = True, inplace = True)
    
    return x

In [ ]:
# Linear Prediction

def linear_prediction(df, pred_col, group_col, month_col = 'month', months = 3):

    # convert to datetime and sort df
    x = df.copy()
    x[month_col] = pd.to_datetime(x[month_col])
    x.sort_values([group_col,month_col], inplace = True)

    # calculate the linear prediction
    x[f'linear_pred_{months}m'] = x.groupby(group_col)[pred_col].shift(1).rolling(window = months, min_periods = months).\
                                  apply(lambda x: LinearRegression().fit(pd.Series(range(len(x))).values.reshape(-1,1), \
                                  x.values.reshape(-1,1)).predict([[months]])[0][0]).round(2).reset_index(level = 0, drop = True)

    x[f'slope_{months}m'] = x.groupby(group_col)[pred_col].shift(1).rolling(window = months, min_periods = months).\
                            apply(lambda x: LinearRegression().fit(pd.Series(range(len(x))).values.reshape(-1,1), \
                            x.values.reshape(-1,1)).coef_[0][0]).round(2).reset_index(level = 0, drop = True)
                           
    # convert back to string
    x[month_col] = x[month_col].dt.strftime('%Y-%m-%d')

    return x

In [ ]:
def add_group_column(df, columns):

    final = pd.DataFrame()
    counter = 0
    lists = [df[col].unique().tolist() for col in columns]
    
    print('\033[1mUnique values :\033[0m')
    print(lists), print('')
    print(f'\033[1mPossible combinations = \033[0m{len(list(itertools.product(*lists))):,}')

    for combination in list(itertools.product(*lists)):
        temp = df.copy()

        for i, _ in enumerate(columns):
            temp = temp[temp[columns[i]] == combination[i]].copy()
            temp['group_name'] = ' | '.join(map(str, combination))

            final = pd.concat([final,temp], ignore_index = True)
        counter += 1 if len(temp) > 0 else 0

    final.sort_values(by = columns, ignore_index = True, inplace = True)
    print(f"\033[1mPopulated combinations = \033[0m{final['group_name'].nunique():,}")
    print('')

    return final

#x = add_group_column(df, ['gender','category','payment_method'])
#x.head(3)

In [ ]:
# How to Choose the Number of Clusters:
# https://www.youtube.com/watch?v=FqIGui0rwh4

def elbow_plot(df, col_name, num_clusters = 10):

    rcParams['figure.figsize'] = 10,5

    values_reshape = df.loc[:,col_name].values.reshape(-1, 1)

    wcss = []
    for i in range(1, num_clusters+1):
        kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
        kmeans.fit(values_reshape)
        wcss.append(kmeans.inertia_)
        
    plt.plot(range(1, num_clusters+1), wcss, marker = 'o', ms = 10, lw = 1)
    plt.title('The Elbow Method', weight = 'bold')
    plt.xlabel('Number of clusters')
    plt.ylabel('WCSS')
    plt.gca().xaxis.set_major_locator(plt.MultipleLocator(1));

In [ ]:
# K-means 

def k_means(df, column, num_clusters = 5, quan = .99, name = ''):

    # exclude outliers from the algorithm
    if quan is not None:
        threshold = df[column].quantile(quan)
        new = df[df[column] < threshold].copy()
        outliers = df[df[column] >= threshold].copy()
    else:
        new = df.copy()

    # execute K-means
    kmeans = KMeans(n_clusters = num_clusters, init = 'k-means++', random_state = 42)
    values_reshape = new.loc[:,column].values.reshape(-1,1)
    y_kmeans = kmeans.fit_predict(values_reshape)

    values = new.loc[:,column].copy()
    temp = pd.DataFrame({'y_kmeans':y_kmeans, 'values':values})

    # assign borders
    borders = []
    for i in range(num_clusters):
        min_border = temp[temp.y_kmeans == i]['values'].min()
        borders = np.append(borders, min_border)
    borders.sort()

    new_col = 'cluster' if name == '' else f'{name}_cluster'
    new[new_col] = None
    for i, v in enumerate(borders, start = 1):
        new[new_col] = np.where(new[column] >= v, i, new[new_col])
        
    # add outliers    
    if quan is not None:
        outliers[new_col] = num_clusters
        new = pd.concat([new,outliers], ignore_index = True)

    print(new.groupby(new_col)[column].agg(['count','min','max']))     

    return new

In [ ]:
# Groupby with array

def group_with_array(df, groupby_col, array_col):

    def to_array(x):
        if len(x) > 1:
            x = np.unique(x)
            x.sort()
        else:
            x = np.array(x)
        return x

    def array_to_string(arr):
        return ', '.join(map(str, arr))
        
    # generate the new structure    
    final = df.groupby(groupby_col)[array_col].agg(to_array).reset_index().reindex([groupby_col,array_col], axis = 1)
    final[f'{array_col}_str'] = final[array_col].apply(array_to_string)
    final['num_values'] = final[array_col].str.len().fillna(0).astype(int)
    
    return final

In [ ]:
# Get quantiles

def get_quantile(df, groupby_cols, quantile_col, num_quantiles = 10, print_summary = True):

    final = pd.DataFrame()
    new_col = quantile_col + ' | quantile'

    lists = [df[col].unique().tolist() for col in groupby_cols]

    for combination in list(itertools.product(*lists)):
        t = df.copy()

        for i, _ in enumerate(groupby_cols):
            t = t[t[groupby_cols[i]] == combination[i]].copy()

        if len(t[quantile_col]) <= num_quantiles:
            t[new_col] = 1
        else:
            t[new_col] = (pd.qcut(t[quantile_col], num_quantiles, labels = False, duplicates = 'drop') + 1)

        final = pd.concat([final,t], ignore_index = True)

    final.sort_values(by = groupby_cols + [new_col], ignore_index = True, inplace = True)
    
    if print_summary == True:
        print(final.groupby(groupby_cols + [new_col])[quantile_col].agg(['count','min','max','mean','median']).round(2))

    return final

In [ ]:
# Text similarity score ("fuzzywuzzy")

strings_to_drop = ['co','pte','ltd','inc','gmbh','limited','financial','consulting',
                   'group','global','digital','international','securities','technology',
                   'technologies','previous'] + list(string.punctuation)

def calc_similarity(row, col1, col2):
    
    def clean_string(s):
        if strings_to_drop:
            for string_to_drop in strings_to_drop:
                s = s.replace(string_to_drop, '').strip().lower()
                s = replace_accents(s) 
        return s
    
    cleaned_col1 = clean_string(row[col1])
    cleaned_col2 = clean_string(row[col2])

    if pd.isna(cleaned_col1) or pd.isna(cleaned_col2):
        return 0
    else:
        return fuzz.ratio(cleaned_col1, cleaned_col2)
    
#df['similarity'] = df.apply(calc_similarity, axis = 1, args = (col1, col2))

In [ ]:
def extract_jira(from_date, to_date, jira_url, jira_username, jira_api_token):

    def adjust_column(df, col_name = 'reporter'):
        df[col_name] = df[col_name].str.title().str.strip()
        df = df.applymap(replace_accents)

    session = requests.Session()
    session.auth = (jira_username, jira_api_token)
    
    start_at, max_results = 0, 100
    issues = []
    
    while True:
        jql_query = f'project = DATA AND created >= {from_date} AND created <= {to_date} ORDER BY created DESC'
        
        params = {'jql':jql_query, 'startAt':start_at, 'maxResults':max_results}
        
        response = session.get(f'{jira_url}search', params = params)
        
        if response.status_code == 200:
            batch_issues = response.json()['issues']
            issues.extend(batch_issues) 
    
            if len(batch_issues) < max_results:
                break
    
            start_at += max_results
        else:
            print(f'Error: {response.status_code} - {response.text}')
            break
    
    key,summary,status,created,sprint,reporter,email,assignee,team,rapyd_bu,department,story_points,issue_type = [],[],[],[],[],[],[],[],[],[],[],[],[]
    
    for issue in issues:
        key.append(issue['key'])
        summary.append(issue['fields']['summary'])
        status.append(issue['fields']['status']['name'])
        created.append(issue['fields']['created'])
        sprint.append(issue['fields'].get('customfield_10113', None))
        reporter.append(issue['fields']['reporter']['displayName'] if issue['fields']['reporter'] else None)
        email.append(issue['fields']['reporter']['emailAddress'] if issue['fields']['reporter'] else None)
        assignee.append(issue['fields']['assignee']['displayName'] if issue['fields']['assignee'] else None)
        team.append(issue['fields']['customfield_10703']['value'] if issue['fields']['customfield_10703'] else None)
        rapyd_bu.append(issue['fields']['customfield_11671']['value'] if issue['fields']['customfield_11671'] else None)
        department.append(issue['fields']['customfield_10623']['value'] if issue['fields']['customfield_10623'] else None)
        story_points.append(issue['fields'].get('customfield_10595', None))
        issue_type.append(issue['fields']['issuetype']['name'])
    
    data = {'issue_key':key, 'summary':summary, 'status':status, 'created':created, 'sprint':sprint,
            'reporter':reporter, 'email':email, 'assignee':assignee, 'team':team, 'business_unit':rapyd_bu, 
            'department':department, 'story_points':story_points, 'issue_type': issue_type}
    
    jira = pd.DataFrame(data)
    
    jira['created'] = pd.to_datetime(jira['created'].str.split('T').str[0] + ' 00:00:00')
    jira['month'] = pd.to_datetime(jira.created).dt.to_period('M')

    jira['sprint'] = jira['sprint'].apply(lambda x: x[0]['name'] if isinstance(x,list) and len(x) > 0 and 'name' in x[0] else None)

    #jira['bi_n_analytics_sprint'] = jira.analytics_sprint.str.extract(r'(\D*)(\d.*)')[1].str.replace(' ','')
    
    jira['story_points'] = pd.to_numeric(jira['story_points'])
    adjust_column(jira)

    print(f'Dates: {from_date} until {to_date}')
    print(f'No. of tickets = {jira.issue_key.count():,}')
    print('')

    return jira

In [ ]:
def count_plot(df, col, hue, figsize = (15,3), fontsize = 8, xticks_rot = 90):
    
    plt.style.use('fivethirtyeight')
    plt.rcParams.update({'figure.figsize':figsize,'font.size':fontsize})

    temp = df.sort_values(by = [col,hue]).copy()

    p1 = sns.countplot(data = temp, x = col, hue = hue)

    # annotation (absolute values)
    for p in p1.patches: 
        if p.get_height() > 0:
            anot = int(p.get_height())
            p1.annotate('{:,}'.format(anot), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

    # annotation (share of total)
    a = temp.groupby(col, as_index = False)[hue].count().sort_values(by = col)
    b = np.tile(a.iloc[:,-1].to_numpy(), temp[hue].nunique())
    zeroes = pd.crosstab(df[col],df[hue]).values.T.flatten()
    totals = [v2 for v1,v2 in zip(zeroes,b) if v2 != 0]

    for p, t in zip(p1.patches, totals):
        if p.get_height() > 0:
            anot = f'{round(p.get_height() / t * 100, 1)}%'
            p1.annotate(anot, (p.get_x() + p.get_width() / 2., p.get_height()), weight = 'bold', ha = 'center', va = 'center', xytext = (0, 25), textcoords = 'offset points')        

    plt.legend(bbox_to_anchor = [1.12, 0.6], title = hue)
    plt.title(col.title().replace('_',' '), weight = 'bold', fontsize = 12, pad = 30)

    p1.set_xticklabels(p1.get_xticklabels(), rotation = xticks_rot, ha = 'center')
    p1.set(xlabel = '', ylabel = '')
    plt.show()

In [ ]:
import matplotlib.patches as mpatches
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error

to_float = lambda x: ("%.8f" % x).rstrip('0').rstrip('.')

"""
function's params:
low_q         --> lower percentile for defining outliers (default value = 5%)
high_q        --> higher percentile for defining outliers (default value = 95%)
kde_max_q     --> max percentile for capping the actual values in the KDE plot (default value = 90%) 
max_pct_diff  --> interval's max % change between actual & predicted values
only_pct_diff --> if the predicted value is lower than X, define intervals only by % diff.
"""

def accuracy_scores(df, actual_col, predicted_col, low_q = .05, high_q = .95, kde_max_q = .95, plots = 'yes'):
                  
    # print input data
    print(f'Input dataframe length = {len(df):,}')
    for col in [actual_col, predicted_col]:
        length = len(df[df[col] > 0])
        print(f'Rows with {col.upper()} = {length:,} ({round(length/len(df)*100,2)}% of input)')

    # keep only rows with sufficient data    
    x = df[(df[actual_col] > 0) & (df[predicted_col] > 0)].copy()
    print('')
    print(f'Rows with Sufficient Data = {len(x):,} ({round(len(x)/len(df)*100,2)}% of input)')
       
    # absolute error
    x['absolute_error'] = x[actual_col] - x[predicted_col]
    
    # drop outliers
    low, high = x.absolute_error.quantile(low_q), x.absolute_error.quantile(high_q)
    x['outlier'] = np.where((x.absolute_error < low) | (x.absolute_error > high), 1, 0)

    print(f'Dropped outliers [{int(low_q*100)}%,{int(high_q*100)}%] = {x.outlier.sum().astype(int):,} ({round(x.outlier.sum()/len(x)*100,2)}% of sufficient data, based on absolute error)')
    
    x = x[x.outlier == 0].copy()
    print(f'Remained rows = {len(x):,}')
    
    print('')
    print('-----------------------------------------------------------------------------------------------------')
    print('R square                  | R²    = 1 - Σ(actual - predicted)^2 / Σ(actual - mean(actual))^2')
    print('* R² is the % of variation explained by the relationship between 2 variables')           
    print('Mean Absolute Error       | MAE   = 1/n * Σ|actual - predicted|')
    print('Root Mean Squared Error   | RMSE  = √(1/n * Σ(actual - predicted)^2)')
    print('Mean Abs Percentage Error | MAPE  = 100 / n * Σ[|actual - predicted| / |actual|]')
    print('Symmetric MAPE            | SMAPE = 100 / n * Σ[2 * |actual - predicted| / (|actual| + |predicted|)]')
    print('Median Absolute Error     | MedAE = median(|actual - predicted|)')
    print('-----------------------------------------------------------------------------------------------------')
    print('')
    
    # R-squared
    r2 = r2_score(x[actual_col], x[predicted_col])
       
    # mean absolute error
    mae = mean_absolute_error(x[actual_col], x[predicted_col])
    
    # root mean squared error
    rmse = np.sqrt(mean_squared_error(x[actual_col], x[predicted_col]))

    actual, forecast = x[actual_col].to_numpy(), x[predicted_col].to_numpy()

    # mean absolute percentage error
    def calc_mape(actual, forecast):
        denominator = np.maximum(np.abs(actual), 1e-7)
        diff = np.abs(forecast - actual)
        return 100 / len(actual) * np.sum(diff / denominator)
    mape = calc_mape(actual, forecast)
    
    # symmetric mean absolute percentage error
    def calc_smape(actual,forecast):
        denominator = np.maximum(np.abs(actual) + np.abs(forecast), 1e-7)
        diff = np.abs(forecast - actual)
        return 100 / len(actual) * np.sum(2 * diff / denominator)
    smape = calc_smape(actual, forecast)
    
    # median absolute error
    medae = median_absolute_error(x[actual_col], x[predicted_col])
    
    print('\033[1m' + 'Accuracy Scores :')
    print('\033[0m')
    print(f'R²    = {round(r2*100,2)}%')
    print(f'MAE   = {round(mae,2)}')
    print(f'RMSE  = {round(rmse,2)}')
    print(f'MAPE  = {round(mape,2)}')
    print(f'SMAPE = {round(smape,2)}')
    print(f'MedAE = {round(medae,2)}')
       
    final = pd.DataFrame({'param':['r2','mae','rmse','mape','smape','medae'],
                          'value':[r2,mae,rmse,mape,smape,medae]})
    
    print('')
    quantiles = x[[actual_col,predicted_col,'absolute_error']].describe(percentiles = np.arange(.1,1,.1)).astype(int)
    print(quantiles.iloc[3:,:].T)
    
    rcParams['figure.figsize'] = 10, 6
    plt.rcParams.update({'font.size': 12})
    sns.set(style = 'ticks')
    plt.style.use('seaborn-white')
    
    q95 = pd.concat([df[actual_col],df[predicted_col]]).quantile(0.95)
    
    if plots != 'no':
        
        kde_min = x[actual_col].quantile(kde_max_q)
    
        plt.figure(figsize = (8, 6)) 
        sns.set_theme(style = 'whitegrid')
        sns.regplot(data = df[(df[actual_col] <= q95) & (df[predicted_col] <= q95)], x = 'actual', y = 'predicted', scatter = True, color = 'dodgerblue', line_kws = {'color':'orange'})
        plt.show()
        
        # 2nd plot
        plt.figure(figsize = (8, 6)) 
        colors = ['lightgrey','deepskyblue','violet']
        k3 = sns.kdeplot(data = x[x[actual_col] < kde_min], x = 'absolute_error', lw = 3.5, color = colors[0])
        k4 = sns.kdeplot(data = x[x[actual_col] < kde_min], x = actual_col, lw = 3.5, color = colors[1])
        k5 = sns.kdeplot(data = x[x[actual_col] < kde_min], x = predicted_col, lw = 3.5, color = colors[2])
        
        patch_1 = mpatches.Patch(color = colors[1], label = actual_col.title().replace('_', ' '))
        patch_2 = mpatches.Patch(color = colors[2], label = predicted_col.title().replace('_', ' '))
        patch_3 = mpatches.Patch(color = colors[0], label = 'Absolute Error')
        legend = plt.legend(handles = [patch_1,patch_2,patch_3], loc = 'upper right') 
        plt.title(f'KDE Plot (capped up to {int(kde_max_q*100)}%)', weight = 'bold', pad = 20)
        sns.despine(bottom = True, left = True)
        plt.show()

    return x

In [ ]:
def distribution(df, col, n = 10, fig = (14,4), font = 10, sign = ''):
    
    plt.rcParams.update({'figure.figsize':fig, 'font.size':font})

    a = df[col].describe(np.arange(1/n, 1, 1/n)).iloc[4:-1]
    a.plot(marker = 's', ms = 7, lw = 1, color = 'dodgerblue')
    
    for x,y in enumerate(a):
        plt.annotate(f'{round(y,3)}{sign}', (x,y), textcoords = 'offset points', xytext = (0,10), ha = 'center', color = 'black')
        
    plt.figtext(0.38, -0.04, ha = 'left', fontsize = 10, bbox = {'facecolor':'lightgrey','alpha':0.2,'pad':3,'edgecolor':'black'},
            s = f'n = {df[col].count():,} | min = {round(df[col].min(),3):,} | max = {round(df[col].max(),3):,} | avg = {round(df[col].mean(),3):,}')
    
    plt.title(f"Distribution - {col.replace('_',' ')}", weight = 'bold', fontsize = 14)
    plt.xlabel('')
    sns.despine(bottom = True, left = True)
    plt.show()

In [ ]:
# Convert country code to country name

def country_code_to_name(code):
    if pd.isnull(code):
        return None
    try:
        return pycountry.countries.get(alpha_2 = code).name
    except AttributeError:
        return 'Unknown'

In [ ]:
def smart_groupby(df, groupby_col, value_col, aggfunc = 'sum', threshold = .01, others_name = 'Other', 
                  dropna = False, consolidate_null = False, top_n = False):

    a = df.copy()
    gb = a.groupby(groupby_col, dropna = dropna)[value_col].agg(aggfunc).sort_values(ascending = False)
    
    print(f'Pre  --> {value_col} = {df[value_col].sum().astype(int):,}')
    
    if gb[gb.index.isnull()].shape[0] > 0:
        print(f'Null values = {gb[gb.index.isnull()].values[0]:,}')
    else:
        print('Null values = 0')
    
    if consolidate_null:
        a[groupby_col] = a[groupby_col].fillna(others_name)
    
    if isinstance(top_n,int) and top_n > 0:
        top_values = gb.head(top_n - 1).index
        a[groupby_col] = a[groupby_col].apply(lambda x: x if x in top_values else others_name)
    else:
        threshold_value = gb.sum() * threshold
        consolidated_values = gb[gb < threshold_value].index
        a[groupby_col] = a[groupby_col].apply(lambda x: others_name if x in consolidated_values else x)

    gb2 = a.groupby(groupby_col, dropna = dropna, as_index = False)[value_col].agg(aggfunc).\
                    sort_values(by = value_col, ascending = False, ignore_index = True)
    gb2['pct_total'] = gb2[value_col] / gb2[value_col].sum()
    gb2['cumsum'] = gb2['pct_total'].cumsum()
    
    print(f'Post --> {value_col} = {gb2[value_col].sum().astype(int):,}'), print('')
    
    gb2.index += 1

    return gb2.style.background_gradient(subset = 'pct_total', cmap = 'Blues', axis = None).\
           format({'pct_total':"{:.2%}",'cumsum':"{:.2%}"}, precision = 2)